# Academy of Py Report


## Analysis

* Passing scores for Reading 100% overall in the district.
* Students from Charter schools demonstrate higher overall scores rates.
* Students attending smaller schools show 100% passing scores for math. There seems to be a bias where students perform higher scores in smaller school overall. The larger the school the lower the overall passing rate  

#### Retrieve data from raw CSV files for schools and students. Create data frames for each

In [520]:
# Dependendencies
import pandas as pd
import os

In [521]:
# Raw Schools data
raw_data_file_sch = "schools_complete.csv"
schools = os.path.join("raw_data",raw_data_file_sch)

In [522]:
# get school data into a dataframe
sch_df = pd.read_csv(schools)

In [523]:
# get number of schools
number_of_schools = sch_df['School ID'].count()
# get total budget
budget = sch_df['budget'].sum()

In [524]:
# Raw Students data
raw_data_file_stu = "students_complete.csv"
students = os.path.join("raw_data",raw_data_file_stu)

In [525]:
# get student data into a dataframe
stu_df = pd.read_csv(students)

In [526]:
student_population = stu_df['Student ID'].count() 
# alternative: student_population = len(stu_df)

In [527]:
# averages scores
r_avg = stu_df['reading_score'].mean()
m_avg = stu_df['math_score'].mean()

# passing grade must be >60
r_pass = stu_df.loc[stu_df['reading_score'] > 60,"reading_score"]
read_pass = r_pass.count()
m_pass = stu_df.loc[stu_df["math_score"] > 60,"math_score"]
math_pass = m_pass.count()
r_per = (read_pass/student_population)*100 
m_per = (math_pass/student_population)*100 
pass_rate = (r_per+m_per)/2

# put it all in a dictionay
district_df = pd.DataFrame({'Total Schools':[number_of_schools],
                            'Total Students':[student_population],
                            'Total Budget': ['${:,.2f}'.format(budget)],
                            'Average Math Score':[m_avg], 
                            'Average Reading Score':[r_avg],
                            '% Passing Math':['{:,.2f}%'.format(m_per)],
                            '% Passing Reading':['{:,.2f}%'.format(r_per)],
                            'Overall Passing Rate':['{:,.2f}%'.format(pass_rate)]                             
                            }
                          )

## District Summary

Create a high level snapshot (in table form) of the district's key metrics

In [528]:
district_df

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Rate,Total Budget,Total Schools,Total Students
0,90.91%,100.00%,78.985371,81.87784,95.45%,"$24,649,428.00",15,39170


#### Combine data for students and schools

In [529]:
# aggregate Schools and Students data into one dataframe
sch_df = sch_df.rename(columns={'name':'school'})
sch_stu_df = pd.merge(sch_df,stu_df,on='school',how='outer')

In [530]:
#Students by Schools
bySchool = sch_stu_df.groupby('school')
students_sch = bySchool['Student ID'].count()

#Budget by school
school_budget = bySchool['budget'].unique().astype(int)
school_budget_ft = school_budget.map('${:,.2f}'.format)
budget_per_stu = school_budget/students_sch
budget_per_stu_ft = budget_per_stu.map('${:,.2f}'.format)


#Score averages by School
math_avg = bySchool['math_score'].mean()
read_avg = bySchool['reading_score'].mean()

#School type
school_type = bySchool['type'].unique()

In [531]:
#Math pass percentage
mathpass = sch_stu_df.loc[sch_stu_df['math_score']>60]
mathpass_sch = mathpass['school'].value_counts()
mathpercentage = (mathpass_sch/students_sch)*100

#Reading pass percentage
readpass = sch_stu_df.loc[sch_stu_df['reading_score']>60]
readpass_sch = readpass['school'].value_counts()
readpercentage = (readpass_sch/students_sch)*100

#Overall passing rate
pass_rate = (readpercentage+mathpercentage)/2

#Collect all together
school_dict = {'type':school_type,
               'students':students_sch,
               'budget':school_budget_ft,
               'per student budget':budget_per_stu_ft,
               'Average Math Score':math_avg,
               'Average Reading Score': read_avg,
               '% passing Math':mathpercentage,
               '% passing Reading':readpercentage,
               'Overall passing':pass_rate
              }
schools_df = pd.DataFrame(school_dict)

## School Summary

Create an overview table that summarizes key metrics about each school

In [532]:
schools_df.head()

,% passing Math,% passing Reading,Average Math Score,Average Reading Score,Overall passing,budget,per student budget,students,type
school,,,,,,,,,
Bailey High School,87.439711,100.0,77.048432,81.033963,93.719855,"$3,124,928.00",$628.00,4976,[District]
Cabrera High School,100.000000,100.0,83.061895,83.975780,100.000000,"$1,081,356.00",$582.00,1858,[Charter]
Figueroa High School,86.436080,100.0,76.711767,81.158020,93.218040,"$1,884,411.00",$639.00,2949,[District]
Ford High School,87.221614,100.0,77.102592,80.746258,93.610807,"$1,763,916.00",$644.00,2739,[District]
Griffin High School,100.000000,100.0,83.351499,83.816757,100.000000,"$917,500.00",$625.00,1468,[Charter]


### Top Performing Schools (By Passing Rate)

Create a table that highlights the top 5 performing schools based on Overall Passing Rate.  Include all of the same metrics as above.

In [533]:
top_5 = schools_df.sort_values(by='Overall passing', ascending=False)[:5]
top_5

,% passing Math,% passing Reading,Average Math Score,Average Reading Score,Overall passing,budget,per student budget,students,type
school,,,,,,,,,
Cabrera High School,100.0,100.0,83.061895,83.975780,100.0,"$1,081,356.00",$582.00,1858,[Charter]
Griffin High School,100.0,100.0,83.351499,83.816757,100.0,"$917,500.00",$625.00,1468,[Charter]
Holden High School,100.0,100.0,83.803279,83.814988,100.0,"$248,087.00",$581.00,427,[Charter]
Pena High School,100.0,100.0,83.839917,84.044699,100.0,"$585,858.00",$609.00,962,[Charter]
Shelton High School,100.0,100.0,83.359455,83.725724,100.0,"$1,056,600.00",$600.00,1761,[Charter]


Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Include all of the same metrics as above.

In [534]:
bottom_5 = schools_df.sort_values(by='Overall passing', ascending=True)[:5]
bottom_5

,% passing Math,% passing Reading,Average Math Score,Average Reading Score,Overall passing,budget,per student budget,students,type
school,,,,,,,,,
Figueroa High School,86.436080,100.0,76.711767,81.158020,93.218040,"$1,884,411.00",$639.00,2949,[District]
Rodriguez High School,86.446612,100.0,76.842711,80.744686,93.223306,"$2,547,363.00",$637.00,3999,[District]
Hernandez High School,86.450917,100.0,77.289752,80.934412,93.225458,"$3,022,020.00",$652.00,4635,[District]
Johnson High School,86.704474,100.0,77.072464,80.966394,93.352237,"$3,094,650.00",$650.00,4761,[District]
Huang High School,86.835790,100.0,76.629414,81.182722,93.417895,"$1,910,635.00",$655.00,2917,[District]


### Math Scores by Grade

 Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.



In [535]:
# group by school and grade to obtain the math average
byGrade = sch_stu_df.groupby(['school','grade'])
bySchool_math_df = pd.DataFrame(byGrade['math_score'].mean().unstack())
bySchool_math_df.head()

grade,10th,11th,12th,9th
school,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010


### Reading Scores by Grade

Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [536]:
# groups by school and grade to obtain the reading average
bySchool_reading_df = pd.DataFrame(byGrade['reading_score'].mean().unstack())
bySchool_reading_df.head()

grade,10th,11th,12th,9th
school,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193


### Scores by School Spending

Create a table that breaks down school performances based on average Spending Ranges (Per Student). 

In [537]:
schools_df['per student budget'] = budget_per_stu
topspend = schools_df['per student budget'].max()
bottomspend = schools_df['per student budget'].min()
bottomspend = budget_per_stu.min()

t2 = (topspend - bottomspend)/4
spending = [0, bottomspend + t2, bottomspend+(2*t2), bottomspend+(3*t2), topspend]
spending_labels = ['<$598','$598-616','$616-635','>$635']

school_spending = pd.cut(schools_df['per student budget'], spending, labels=spending_labels)
school_spending

spending_dict = {"Spending":school_spending,
                "Average Math Score":math_avg,
                "Average Reading Score":read_avg,
                "% passing math":mathpercentage,
                "% passing read":readpercentage}

spending_df = pd.DataFrame(spending_dict)
spending_df.groupby('Spending').mean()

,% passing math,% passing read,Average Math Score,Average Reading Score
Spending,,,,
<$598,100.000000,100.0,83.455399,83.933814
$598-616,100.000000,100.0,83.599686,83.885211
$616-635,93.719855,100.0,80.199966,82.425360
>$635,88.585069,100.0,77.866721,81.368774


## Scores by School Size

Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).



In [538]:
#schools_df['students'].unique()
sch_size = [0,1500,3000,5000]
name_size = ['Small(<1500)', 'Medium(1500-3000)','Large(3000-5000)']

school_size = pd.cut(schools_df['students'],sch_size, labels=name_size)
school_size

size_dict = {"Size":school_size,
                "Average Math Score":math_avg,
                "Average Reading Score":read_avg,
                "% passing math":mathpercentage,
                "% passing read":readpercentage}

size_df = pd.DataFrame(size_dict)
size_df.groupby("Size").mean()


,% passing math,% passing read,Average Math Score,Average Reading Score
Size,,,,
Small(<1500),100.000000,100.0,83.664898,83.892148
Medium(1500-3000),95.061685,100.0,80.904987,82.822740
Large(3000-5000),86.760428,100.0,77.063340,80.919864


## Scores by School Type

Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).


In [539]:
type_dict = {"type":school_type,
                "Average Math Score":math_avg,
                "Average Reading Score":read_avg,
                "% passing math":mathpercentage,
                "% passing read":readpercentage
          }
type_df = pd.DataFrame(type_dict)
type_df['type'] = type_df['type'].map("%s".join)
type_df.groupby('type').mean()

,% passing math,% passing read,Average Math Score,Average Reading Score
type,,,,
Charter,100.000000,100.0,83.473852,83.896421
District,86.790742,100.0,76.956733,80.966636
